In [ ]:
%matplotlib inline
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,6)

### Implicit Multithreading in NumPy

#### A Matrix Operation

In [ ]:
n = 20
m = 1000

for i in range(n):
    X = np.random.randn(m, m)
    λ = np.linalg.eigvals(X)

#### A Multithreaded Ufunc

In [ ]:
def f(x, y):

    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)

grid = np.linspace(-3, 3, 5000)
x, y = np.meshgrid(grid, grid)

In [ ]:
%timeit np.max(f(x, y))

#### A Comparison with Numba

In [ ]:
from numba import vectorize

@vectorize
def f_vec(x, y):

    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)

np.max(f_vec(x, y)) # Run once to compile

In [ ]:
%timeit np.max(f_vec(x, y))

#### Multithreading a Numba Ufunc

In [ ]:
@vectorize('float64(float64, float64)', target='parallel')
def f_vec(x, y):

    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)

np.max(f_vec(x, y)) # Run once to compile

In [ ]:
%timeit np.max(f_vec(x, y))

### Multithreaded Loops in Numba

In [ ]:
from numpy.random import randn
from numba import njit

@njit
def h(w, r=0.1, s=0.3, v1=0.1, v2=1.0):
    """
    Updates household wealth.
    """

    # Draw shocks
    R = np.exp(v1 * randn()) * (1 + r)
    y = np.exp(v2 * randn())

    # Update wealth
    w = R * s * w + y

    return w

In [ ]:
fig, ax = plt.subplots()

T = 100
w = np.empty(T)
w[0] = 5

for t in range(T-1):
    w[t+1] = h(w[t])

ax.plot(w)
ax.set_xlabel('$t$', fontsize=12)
ax.set_ylabel('$w_{t}$', fontsize=12)
plt.show()

In [ ]:
@njit
def compute_long_run_median(w0=1, T=1000, num_reps=50_000):
    obs = np.empty(num_reps)

    for i in range(num_reps):
        w = w0
    
        for t in range(T):
            w = h(w)

        obs[i] = w
        
    return np.median(obs)

In [ ]:
%%time
compute_long_run_median()

In [ ]:
from numba import prange

@njit(parallel=True)
def compute_long_run_median_parallel(w0=1, T=1000, num_reps=50_000):
    obs = np.empty(num_reps)
    
    for i in prange(num_reps):
        w = w0
        for t in range(T):
            w = h(w)
        
        obs[i] = w
    
    return np.median(obs)

In [ ]:
%%time
compute_long_run_median_parallel()

### Exercises